In [58]:
import os 
import sys 
import pandas as pd 
import numpy as np 
import re
import pickle
import json
import sqlite3
from pprint import pprint
import datetime

In [59]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
with open('data/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [62]:
os.environ.update(credentials)

In [63]:
scope = "user-read-recently-played"

In [64]:
os.environ['SPOTIPY_REDIRECT_URI'] = 'https://malcolm-taylor.info/'
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

In [65]:
def split_utc_time_str(time_str):
    part1 = datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) - datetime.timedelta(hours=4)
    date = str(part1.date())
    time = str(part1.time().strftime('%H:%M'))
    return(date, time)

In [66]:
def get_recently_played(after=None):
    recently_played = spotify.current_user_recently_played(after=after)
    # Describe json 
    print('Getting songs at ', datetime.datetime.now())
    before = recently_played['cursors']['before']
    after = recently_played['cursors']['after']
    n_items = len(recently_played['items'])
    print(f'There are {n_items} songs between '
          , datetime.datetime.fromtimestamp(int(before)/1000)
          , '  and  '
          , datetime.datetime.fromtimestamp(int(after)/1000))
    
    if n_items == 0:
        return(pd.DataFrame(columns = ['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']))
    
    # Parse recently played output 
    songs_list = []
    wanted_keys = ['name', 'duration_ms', 'popularity', 'uri']
    for song in recently_played['items']:
        temp_song_dict = {}
        temp_song_dict['after_ts'] = after
        temp_song_dict['played_at_date'], temp_song_dict['played_at_time'] = split_utc_time_str(song['played_at'])
        for k in wanted_keys:
            temp_song_dict[k] = song['track'][k]
        
        # Take first artist information 
        temp_song_dict['artist_name'] = song['track']['artists'][0]['name']
        temp_song_dict['artist_id'] = song['track']['artists'][0]['uri']
        temp_song_dict['playlist_id'] = song['context']['uri']
        
        songs_list.append(temp_song_dict)
    
    # Export cleanly 
    out_series = pd.DataFrame(songs_list)
    out_series['duration_min'] = out_series['duration_ms']/60000.0
    out_series = out_series.rename({'uri':'song_uri'}, axis=1) 
    out_series = out_series[['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id', 'after_ts']]
    
    return(out_series)

In [67]:
db_location = 'listening_history.db'

In [68]:
con = sqlite3.connect(db_location)

In [72]:
cursor = con.cursor()

In [73]:
rslt = cursor.execute('select max(after_ts) from Listening_History').fetchone()
latest_time_pull = rslt[0]
print("Latest pull from ", 
datetime.datetime.fromtimestamp(int(latest_time_pull)/1000)) 

Latest pull from  2020-08-01 01:41:20.778000


In [75]:
newly_played = get_recently_played(latest_time_pull)

Getting songs at  2020-08-01 10:03:27.895104
There are 11 songs between  2020-08-01 01:47:10.880000   and   2020-08-01 02:50:52.937000


In [76]:
newly_played.head()

,name,artist_name,played_at_date,played_at_time,duration_min,popularity,song_uri,artist_id,playlist_id,after_ts
0,Isla De Flores,Berlioz,2020-08-01,02:50,2.652067,68,spotify:track:7tDHMhO0bJpbkXzMyMIE0b,spotify:artist:6kndrupH2JaLYqh1wBKGar,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
1,Alene uden dig,Tosh Møller,2020-08-01,02:33,2.625000,66,spotify:track:3eMTcFDfBkdUxzRDAb8Krs,spotify:artist:2CUZliHXKYjXEV2BOZqMvi,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
2,Sand Between Our Fingers,Comet Blue,2020-08-01,02:30,2.802083,67,spotify:track:55S0Xumnc0oNtM3fzh2Oex,spotify:artist:27DGFhdWUeudNFbPJROkWF,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
3,Vals De Rodrigues,Eric Badanti,2020-08-01,02:27,2.686817,67,spotify:track:3I3WY0C7AQMe6Z6hMUQqd5,spotify:artist:2KnoW7Kuw3lj3OKFK8EXQ7,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
4,Choices,Patricia Sevin,2020-08-01,02:24,1.978567,67,spotify:track:4ynA4JLwQIRfIhPuOLVoHZ,spotify:artist:7fJ7NnyfXlucXl3E3Q92Tt,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937


In [77]:
newly_played.to_sql('Listening_History', con, if_exists='append' )

In [78]:
con.commit()
con.close()

In [79]:
newly_played

,name,artist_name,played_at_date,played_at_time,duration_min,popularity,song_uri,artist_id,playlist_id,after_ts
0,Isla De Flores,Berlioz,2020-08-01,02:50,2.652067,68,spotify:track:7tDHMhO0bJpbkXzMyMIE0b,spotify:artist:6kndrupH2JaLYqh1wBKGar,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
1,Alene uden dig,Tosh Møller,2020-08-01,02:33,2.625000,66,spotify:track:3eMTcFDfBkdUxzRDAb8Krs,spotify:artist:2CUZliHXKYjXEV2BOZqMvi,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
2,Sand Between Our Fingers,Comet Blue,2020-08-01,02:30,2.802083,67,spotify:track:55S0Xumnc0oNtM3fzh2Oex,spotify:artist:27DGFhdWUeudNFbPJROkWF,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
3,Vals De Rodrigues,Eric Badanti,2020-08-01,02:27,2.686817,67,spotify:track:3I3WY0C7AQMe6Z6hMUQqd5,spotify:artist:2KnoW7Kuw3lj3OKFK8EXQ7,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
4,Choices,Patricia Sevin,2020-08-01,02:24,1.978567,67,spotify:track:4ynA4JLwQIRfIhPuOLVoHZ,spotify:artist:7fJ7NnyfXlucXl3E3Q92Tt,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
5,Gaze,Moux,2020-08-01,02:22,2.669333,68,spotify:track:2XeZi7qwChzLOt32tsoPXV,spotify:artist:4jsMhXef3DBlwIfiVLEGDZ,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
6,Onthou,Ever So Blue,2020-08-01,02:20,2.138883,69,spotify:track:0CoSBNNeO8JgayAfLttECk,spotify:artist:6LwcYWG1molG0KW2xwnPd2,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
7,Sea Change,Stephan Moccio,2020-08-01,02:18,2.853550,69,spotify:track:3pJnfyKoVePwVmljB6Wun1,spotify:artist:25s9H1JQmTu3iuFzpXWUIg,spotify:playlist:37i9dQZF1DX4sWSpwq3LiO,1596264652937
8,Silencio,Miguel Alexis,2020-08-01,02:06,4.123633,0,spotify:track:6XQbyK9SKbaLJzB3wQMOsF,spotify:artist:70hnGySo55PprqDnOVwdG9,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1596264652937
9,Mi Media Mitad,Rey Ruiz,2020-08-01,01:52,4.877550,65,spotify:track:64QyipABOzDwoUFqaEnCvU,spotify:artist:1S0uvW1rgvtkNGma9su3fJ,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1596264652937
